In [43]:
import pandas as pd

#Load data
data = pd.read_excel("GooglePlaystore.xlsx")

#Remove invalid records
data = data[data['Reviews'] != '3.0M']
data = data[~data.apply(lambda x: x.astype(str).str.contains('Varies with device')).any(axis=1)]

#Process Android version and Installs
data['Android Ver'] = data['Android Ver'].str.extract(r'(\d+\.\d+)')
data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True).astype(int)

#Convert 'Reviews' to integer for processing
data['Reviews'] = data['Reviews'].astype(int)

def fill_missing_ratings(row):
    if pd.isna(row['Rating']):
        if row['Reviews'] < 100 and row['Installs'] < 50000:
            return None
        else:
            return category_averages[row['Category']]
    return row['Rating']

#Calculate average ratings per category rounded to two decimal places
category_averages = data.groupby('Category')['Rating'].mean().round(2)

data['Rating'] = data.apply(fill_missing_ratings, axis=1)
data = data.dropna(subset=['Rating'])

#Convert 'Size' to integer
def convert_size(size):
    if 'M' in size:
        return int(float(size.replace('M', '')) * 1000000)
    elif 'k' in size or 'K' in size:
        return int(float(size.replace('k', '').replace('K', '')) * 1000)
    else:
        return None  

data['Size'] = data['Size'].apply(convert_size)
data = data.dropna(subset=['Size'])

data.to_excel("updatedGooglePlaystore.xlsx", index=False, engine='openpyxl')

def stats():
    x = categoryRatingStats = data.groupby('Category')['Rating'].describe()
    print(x)
    
stats()

#iso free apps
freeApps = data[data['Type'] == 'Free']

#get top 3 apps of a column
def getTop3HelperFunc(group, column_name):
    return group.nlargest(3, column_name)[['Category', 'App', column_name]]

#get top 3 apps using the previous function to actually get the thang
def top3Apps(dataframe, column_name):
    topApps = dataframe.groupby('Category').apply(getTop3HelperFunc, column_name=column_name)
    topApps.reset_index(drop=True, inplace=True)
    return topApps

topRatedApps = top3Apps(freeApps, 'Rating')
topInstalledApps = top3Apps(freeApps, 'Installs')
topReviewedApps = top3Apps(freeApps, 'Reviews')

print(f"Top Rated Free Apps: {topRatedApps.head()}")
print(f"\nTop Installed Free Apps: {topInstalledApps.head()}")
print(f"\nTop Reviewed Free Apps: {topReviewedApps.head()}")

paidApps = data[data['Type'] == 'Paid']
averagePrice = paid_apps['Price'].mean()
maxPrice = paid_apps['Price'].max()
minPrice = paid_apps['Price'].min()
print(f"Average Price of Paid Apps: ${averagePrice:.2f}")
print(f"Maximum Price of Paid Apps: ${maxPrice:.2f}")
print(f"Minimum Price of Paid Apps: ${minPrice:.2f}")

Top Rated Free Apps:             Category                                                App  \
0     ART_AND_DESIGN            Spring flowers theme couleurs d t space   
1     ART_AND_DESIGN                         Harley Quinn wallpapers HD   
2     ART_AND_DESIGN                                  Cardi B Wallpaper   
3  AUTO_AND_VEHICLES  Tickets SDA 2018 and Exam from the State Traff...   
4  AUTO_AND_VEHICLES                     CDL Practice Test 2018 Edition   

   Rating  
0     5.0  
1     4.8  
2     4.8  
3     4.9  
4     4.9  

Top Installed Free Apps:             Category                                                App  \
0     ART_AND_DESIGN                     Tattoo Name On My Photo Editor   
1     ART_AND_DESIGN                                       ibis Paint X   
2     ART_AND_DESIGN  Canva: Poster, banner, card maker & graphic de...   
3  AUTO_AND_VEHICLES                      AutoScout24 - used car finder   
4  AUTO_AND_VEHICLES                        Used cars f